# MAP572 - TD1

This lab's aim is to solve numerically systems of linear equations. In a first part we use an LU decomposition, in a second part we look into gradient descent, and try to optimise how we take our "step".

In [1]:
import numpy as np
import scipy.sparse as sparse
import sys
import time
import matplotlib.pyplot as plt

In [2]:
def triangulaireSup(L,b):
#  This algorithm solves a linear system with an upper triangular matrix. We are looking for x such that L * x = b

# Input:
#      L is an upper triangular matrix of size n*n
#      b is a vector of size n

# Output:
#      vector x of size n

############################


    n = len(b)
    x = np.zeros(n)
    for i in range(1,n+1):
        if L[n-i][n-i] == 0:
            return "Erreur, 0 sur la diagonale à la ligne {}".format(n-i) # make sure our matrix has no 0s on its diagonal
    
        else:
            x[n-i] = (b[n-i] - np.dot(L[n-i][n-i+1:], x[n-i+1:]))/L[n-i][n-i] #calculate x[n-i] depending on all the previous terms calculated
    return x


In [3]:
S = np.array([[1,2,3],[0,1,2],[0,0,1]])
b = np.array([3,2,1])

In [4]:
triangulaireSup(S,b)

array([0., 0., 1.])

In [5]:
def triangulaireInf(L,b):
#  This algorithm solves a linear system with a lower triangular matrix. We are looking for x such that L * x = b

# Input:
#      L is a lower triangular matrix of size n*n
#      b is a vector of size n

# Output:
#      vector x of size n

############################

    n = len(b)
    x = np.zeros(n)
    for i in range(0,n):
        if L[i][i] == 0:
            return "Erreur, 0 sur la diagonale à la ligne {}".format(i) # make sure our matrix has no 0s on its diagonal
    
        else:
            x[i] = (b[i] - np.dot(L[i][:i], x[:i]))/L[i][i] #calculate x[ni] depending on all the previous terms calculated
    return x

In [6]:
I = np.array([[0.5,0,0],[1,2,0],[1,2,3]])

In [7]:
S.shape[0]

3

In [8]:
def factLU(A):
# This algorithm does the LU decomposition of a given matrix

# Input:
#      A is a square matrix of size n

# Output:
#       U is an upper triangular matrix of size n
#       L is a lower triangular matrix of size n

###############################################


    n=np.shape(A)[0]
    L = np.identity(n)
    U = np.zeros(shape=(n,n))
    
    
    for i in range(0, n-1):
        for j in range(i+1, n):
            L[j,i] = A[j,i]/A[i,i]
            
            for k in range(i + 1, n):
                A[j:k] -= L[j, i] * U[i, k]
                
        for j in range(i, n):
            U[i,j] = A[i,j]
            
    U[-1,-1] = A[-1,-1]
    
    return L, U

In [9]:
factLU(I)

(array([[1., 0., 0.],
        [2., 1., 0.],
        [2., 1., 1.]]), array([[0.5, 0. , 0. ],
        [0. , 2. , 0. ],
        [0. , 0. , 3. ]]))

# Matrices creuses

Here we will look at sparse matrixes. We will look at how their sparcity impacts computation time, memory, etc.

In [10]:
def matriceCreuse(N, d):
    
# Algorithm that creates a random sparse matrix using built-in sparse library

# Input:
#      N is the wanted size
#      d is the density of the wanted matrix 

#  Output:
#      A random sparse matrix of size N*N and with a density of d

#####################


    matrix = sparse.rand(N,N, density = d)
    return matrix
   

In [11]:
def espaceDeStockage(N):

# Algorithm that compares the memory space for a regular matrix compared to a sparse one with same size

# Input:
#      N, size of the matrix

# Output:
#      String comparing both sizes

########################################


    A = np.array((N,N))
    taillePleine = sys.getsizeof(A)
    
    B = sparse.rand(N,N, density = 0.01)
    tailleCreuse = sys.getsizeof(B)
    
    return( "tailleCreuse = {}, taillePleine = {}".format(tailleCreuse, taillePleine))



In [12]:
espaceDeStockage(100)

'tailleCreuse = 56, taillePleine = 104'

In [22]:
def comparaisonDesTemps():
#     In this section, we look at the computation time when doing three different operations:
# - multiplying two matrixes
# - multiplying a matrix and a vector
# - multiplying a sparse matrix and a vector

# In order to get the time needed for the computation, we look at the absolute time before the operation and after the operation
# and take the difference. Each function below corresponds to the situations described above.

# The matrixes and vectors used are defined in the cell below

################################


    print("Matrice * Matrice = {} s".format(tempsMatriceMatrice()))
    print("Matrice * Vecteur = {} s".format(tempsMatriceVecteur()))
    print("Creuse * Vecteur = {} s".format(tempsCreuseVecteur()))

def tempsMatriceMatrice():
    t0 = time.time()
    A.dot(B)
    t1 = time.time()
    return t1-t0

def tempsMatriceVecteur():
    t0 = time.time()
    A.dot(vecteur)
    t1 = time.time()
    return t1-t0

def tempsCreuseVecteur():
    t0 = time.time()
    matrix.dot(vecteur)
    t1 = time.time()
    return t1-t0

In [27]:
A = np.random.rand(5000,5000)
B = np.random.rand(5000,5000)
vecteur = np.random.rand(5000)
matrix = sparse.rand(5000, 5000, 0.001)

In [28]:
comparaisonDesTemps()

Matrice * Matrice = 2.733672618865967 s
Matrice * Vecteur = 0.010290861129760742 s
Creuse * Vecteur = 0.0030236244201660156 s


We can observe that the third operation goes roughly ten times faster than the second. the use of sparse matrixes seems to be useful in order to gain  computation time.

# Methodes de type gradient

In this part, we will consider a sparse matrix A. We try to solve the equation A * x = b by different methods of gradient descent. 

In [42]:
def GPF(A,b, tol, alpha, itermax, x0):
# First algorithm uses graient descent with a constant step alpha

#Input:
#     A: a sparse matrix of size n*n
#     b: a vector of size n
#     tol: a float which represents our tolerance
#     alpha: our step, float
#     x0: a vector of size n, our starting point for finding x

#Output:
#    x: the last vector found before we either go below our tolerance, or go above the max iterations we gave
#    iterations: if we arrive below tolerance, the number of iterations that were necessary. Otherwise, itermax


    r = b - A.dot(x0)
    x = x0 + alpha*r
    iterations = 0
    while (np.linalg.norm(r) > tol):
        if (iterations > itermax):
            print("Max iterations overflow")
            return x, iterations
        
        r = b - A.dot(x)
        x = x + alpha * r
        iterations +=1
    
    return x, iterations

In [47]:
sparse_matrix = sparse.rand(10, 10, 0.1)
print(sparse_matrix)

  (8, 8)	0.5286843024355987
  (4, 2)	0.5852939245431306
  (2, 5)	0.4742558623220301
  (6, 2)	0.33803209525476285
  (9, 6)	0.8712265094209549
  (4, 4)	0.47317249364849623
  (6, 6)	0.302249184595323
  (0, 4)	0.5179733979891215
  (4, 6)	0.23479250349674108
  (8, 2)	0.03786691481058169


In [45]:
GPF(sparse_matrix, np.ones(10), 0.001, 0.001, 5000, np.ones(10))

Max iterations overflow


(array([  6.002     ,   6.002     ,   6.002     ,   1.94837548,
          4.09153901,   4.73040051, -10.20796342,   6.002     ,
          4.50949089,  -7.07192677]), 5001)

In [17]:
def nombreIterations():
    X=[]
    Y=[]
    for alpha in np.arange(0.0001, 0.1, 0.001):
        X.append(alpha)
        Y.append(GPF(np.ones((3,3)), [5,4,1], 0.001, alpha, 100000, np.ones(3))[1])
        
    plt.plot(X,Y)
    plt.xscale("log")
    plt.xlabel("alpha - logscale")
    plt.ylabel("iterations")

In [18]:
def GPO(A, b, tol, itermax, x0):
    r = b - np.dot(A,x0)
    alpha = np.dot(r,r)/np.dot(np.dot(A,r), r)
    x = x0 + alpha*r
    iterations = 0
    while (np.linalg.norm(r) > tol):
        if (iterations > itermax):
            print("Max iterations overflow")
            return x, iterations
        
        r = b-np.dot(A,x)
        alpha = np.dot(r,r)/np.dot(np.dot(A,r), r)
        x = x + alpha * r
        iterations +=1
    
    return x, iterations

In [25]:
GPO(np.ones((3,3)), [2,1,1], 0.001, 5000, np.ones(3))

Max iterations overflow


(array([ 8104.24000003, -4050.62000002, -4050.62000002]), 5001)

In [ ]:
aefi,ozc
cizneo